# TCGA Fusion & Preprocessing Notebook 🧬

### Overview 🌟
This notebook is designed to preprocess and integrate multiple datasets, including:

- **Expression matrices** 🧪
- **Clinical data** 🏥
- **Sample metadata** 🗂️

The goal is to merge these datasets into a unified format suitable for training a **Deep Learning Model (MLP)**. 🚀

### Why are we doing this? 🤔

1. **Deep Learning Models**, such as Multi-Layer Perceptrons (MLPs), require structured and clean input data. This ensures the model can learn meaningful patterns without being hindered by inconsistencies or missing values.
2. By combining clinical, expression, and sample data, we create a **comprehensive dataset** that captures both molecular and clinical features. This enables the model to:
   - Predict outcomes more accurately. 🎯
   - Identify key biomarkers. 🔬
   - Support personalized medicine approaches. 💊

### What will this notebook achieve? ✅
- Preprocess raw data files to ensure consistency.
- Map and align identifiers across datasets.
- Merge all relevant features into a single dataset ready for **MLP training**.

Let’s get started! 🚀

In [ ]:
#Librairies and Paths

import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import json

# Paths (edit these)

EXPR_PATH = "C:\\Users\\assou\\Documents\\PYTHON\\BIP12\\gdc_download_20251125_142547.268493"
CLIN_PATH = "../data/clinical.tsv"
SAMPLES_PATH = "../data/sample.tsv"

In [ ]:
#Data Loading

clinical = pd.read_csv(CLIN_PATH, sep="\t")
samples = pd.read_csv(SAMPLES_PATH, sep="\t")

In [ ]:
# Concatenate expression data since each file represents ~ one sample 
# takes around 1 minute to run

expression = []

for root, dirs, files in os.walk(EXPR_PATH):
    for f in files:
        if f.endswith(".tsv"):
            path = os.path.join(root, f) # full path to the file

            try:
                df = pd.read_csv(path, sep="\t", comment="#")
                sample_id = f.split(".")[0]   # sample name based on filename

                # Add only the expression column for this sample
                expression.append(
                    df.set_index("gene_id")[["fpkm_uq_unstranded"]]
                    .rename(columns={"fpkm_uq_unstranded": f})
                )

            except Exception:
                pass

# Concatenate all expression columns horizontally
expr_matrix = pd.concat(expression, axis=1)
expr_matrix = expr_matrix[4:].T.copy()
expr_matrix

## Downloading the Expression Matrix 💾

You can download the **expression matrix** as it is using the following code snippet. However, it is **highly recommended** to preprocess the data first to ensure better quality for downstream analysis. 🧹

### Why preprocess the data? 🤔
- **Filtering**: Remove irrelevant or noisy data.
- **Normalization**: Ensure consistency across samples.
- **Feature selection**: Focus on the most informative genes.

Preprocessing will help improve the performance and reliability of your machine learning models. 🚀

In [ ]:
#expr_matrix.to_csv("../data/TCGA_expression_matrix.tsv", sep="\t")

## Mapping File Names to Entity Submitter IDs 🗂️

In this section, we will:

- 🔗 Create a **mapping dataframe** using the metadata file provided by TCGA.
- 🗃️ Map **`file names`** to **`entity submitter IDs`**.
- 📋 Include other relevant IDs required for merging with clinical data later on.

### Why is this important? 🤔
This mapping is a **crucial step** in preparing the data for:
- Accurate integration of clinical and expression datasets.
- Ensuring consistency across all data sources for downstream analysis. 🚀

In [ ]:
#mapping from metadata case_id to case submitter_id
with open("../data/metadata.cart.2025-11-25.json") as f:
    meta = json.load(f)

rows = []

for entry in meta:
    file_name = entry["file_name"]
    file_id = entry["file_id"]
    
    ent = entry["associated_entities"][0]
    
    aliquot_id = ent["entity_id"]
    case_id = ent["case_id"]
    submitter_id = ent["entity_submitter_id"].split("-")[0:3]
    submitter_id = "-".join(submitter_id)
    
    rows.append({
        "file_name": file_name,
        "aliquot_id": aliquot_id,
        "case_id": case_id,
        "submitter_id": submitter_id
    })

map_df = pd.DataFrame(rows)

#Use this command to download the .csv file of the mapping dataframe (dictionnary)
#map_df.to_csv("metadata_mapping.csv", index=False)

map_df

## Mapping File Names to Submitter IDs in the Expression Matrix 🗂️

This section focuses on:

- 🔗 Mapping **`file names`** to **`submitter IDs`** within the expression matrix.
- ✅ Ensuring that the expression data is correctly aligned with the clinical and sample metadata.

This mapping is a critical step for facilitating accurate downstream analysis and integration. 🚀

In [ ]:
def get_sub_id(filename):
    match = map_df[map_df["file_name"] == filename]
    if len(match) > 0:
        return match["submitter_id"].values[0]
    return filename  # fallback to filename if not found

expr_matrix.index = expr_matrix.index.map(get_sub_id)
expr_matrix.index.name = "submitter_id"

expr_matrix.to_csv("../expression_matrix_sub.tsv", sep="\t", index=True)
expr_matrix

#You should now have an expression matrix with submitter_ids as row indices.

# Adding Clinical and Sample Labels 🏷️

## Second Mapping: Transforming `case_id` to `submitter_id` in Clinical Data

In this step, we will use the mapping dataframe (`map_df`) to:

- Replace **`case_id`** values in the clinical data with their corresponding **`submitter_id`** values from the metadata.

This transformation ensures consistency between the clinical data and the metadata, which is essential for accurate data integration and analysis. 🔄

In [ ]:
print("Colonnes disponibles dans clinical :", clinical.columns.tolist())
#set case_id as index
clinical = clinical.set_index("cases.case_id")
#Map des case_id en submitter_id dans clinical
def get_sub_id_case(filename):
    match = map_df[map_df["case_id"] == filename]
    if len(match) > 0:
        return match["submitter_id"].values[0]
    return filename  # fallback to filename if not found

clinical.index = clinical.index.map(get_sub_id_case)
clinical.index.name = "submitter_id"

clinical.to_csv("../clinical_sub.tsv", sep="\t", index=True)
clinical

#You should now have clinical data with submitter_ids as row indices.

## Sample Data 🧪

This step is crucial for ensuring that all datasets are aligned and can be effectively merged for comprehensive analysis. ✅

### Key Notes:
- In the **sample data**, the `submitter_id` is already present. 🎯
- Therefore, no additional mapping is required. 🚫
- The goal here is simply to set the `submitter_id` as the index for easier access and consistency. 📋

In [ ]:
# Set les cases.submitter_id comme index
samples = samples.set_index("cases.submitter_id")
samples.head()

In [ ]:
# Let's recheck the clinical data to ensure all case_ids have been mapped correctly.
clinical.head()

# Merging Clinical, Expression, and Sample Data 🔗

In this step, we will:

- 🧬 Combine **clinical data**, **expression data**, and **sample data** into a unified dataset.
- 📊 Ensure all datasets are properly aligned using `submitter_id` as the common key.

This fusion is essential for creating a comprehensive dataset ready for downstream analysis and machine learning tasks. 🚀

In [ ]:
final = expr_matrix \
    .merge(clinical[["cases.primary_site", "demographic.age_at_index","diagnoses.ajcc_pathologic_stage",
                     "diagnoses.ajcc_pathologic_t","diagnoses.classification_of_tumor","diagnoses.primary_diagnosis",
                     "treatments.treatment_or_therapy", "treatments.treatment_type"]], 
                     left_index=True, right_index=True, how="inner") \
    .merge(samples[["samples.sample_type","samples.tissue_type"]], left_index=True, right_index=True, how="left")

final

In [ ]:
#changer les gene_ids en gene_names
gene_map = pd.read_csv("../data/gene_name.tsv", sep="\t")
gene_map.head()
gene_dict = pd.Series(gene_map.gene_name.values, index=gene_map.gene_id).to_dict()
final = final.rename(columns=gene_dict)

final

# Download the Final Dataset 💾

### File Name: `final_TCGA_thyroid_data.csv`

⚠️ **Important Notes:**
- The final dataset can be **very large** depending on the number of samples and features included. 📊
- It is recommended to apply **filtering** before downloading if you only need specific subsets of the data. ✂️

The filtering section follows, and you can adapt it based on the type of analysis you want to perform. 🔍
I also recommend saving the final dataset in a compressed format (e.g., `.zip` or `.gz`) to reduce file size and facilitate easier sharing and storage. 📦

To view the dataset, I use the extension "Data Wrangler" in VSCode, which allows for easy exploration of large CSV files, and gives you a summary of each column. 📂

In [ ]:
#Filtering
# For example, filter for thyroid cancer samples only
#final = final[final["diagnoses.classification_of_tumor"] == "Thyroid Carcinoma"]

#You can filter based on the treatment type :
#final = final[final["treatments.treatment_type"] == "Chemotherapy"]

#Filtering based on the sample type :
#final = final[final["samples.sample_type"] == "Primary Tumor"]

#Filtering based on the stage :
#final = final[final["diagnoses.ajcc_pathologic_stage"] == "Stage I"]

#filtering based on age : 
#final = final[final["demographic.age_at_index"] >= 50]

#final.head()

In [ ]:
#final.to_csv("../data/final_TCGA_thyroid_data.csv", sep="\t", index=True)